In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import torch.nn.functional as F

from torch.utils.data import Dataset
torch.manual_seed(0)
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')


In [2]:
# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Create dataset
train_dataset = MyDataset(X_train, y_train)
# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [3]:
# Simpler model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(65, 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*240, 65)  
        self.fc3 = nn.Linear(65, 2)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer

    def forward(self, x):
        x = x.view(-1, 65, 256, 240)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
# Initialize the network and print its architecture
model = Net()
print(model)

Net(
  (conv1): Conv2d(65, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=30720, out_features=65, bias=True)
  (fc3): Linear(in_features=65, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [4]:
# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight = torch.tensor([0.5,1.0]).to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Initialize empty lists to store losses
train_losses = []

# Training loop
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels = labels.type(torch.LongTensor)   # casting to long
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        l1_lambda = 0.0005
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = criterion(outputs, labels)
        loss+=l1_lambda*l1_norm
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

# Now without regularization
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Training loop
for epoch in range(20):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels = labels.type(torch.LongTensor)   # casting to long
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 3.0144
Epoch 2, training loss: 2.0268
Epoch 3, training loss: 1.3780
Epoch 4, training loss: 1.0252
Epoch 5, training loss: 0.8956
Epoch 6, training loss: 0.8356
Epoch 7, training loss: 0.8230
Epoch 8, training loss: 0.8027
Epoch 9, training loss: 0.8040
Epoch 10, training loss: 0.8018
Epoch 11, training loss: 0.7677
Epoch 12, training loss: 0.7583
Epoch 13, training loss: 0.7618
Epoch 14, training loss: 0.7770
Epoch 15, training loss: 0.7990
Epoch 1, training loss: 0.7308
Epoch 2, training loss: 0.6901
Epoch 3, training loss: 0.6770
Epoch 4, training loss: 0.6147
Epoch 5, training loss: 0.5677
Epoch 6, training loss: 0.4948
Epoch 7, training loss: 0.4114
Epoch 8, training loss: 0.3996
Epoch 9, training loss: 0.3288
Epoch 10, training loss: 0.2399
Epoch 11, training loss: 0.2265
Epoch 12, training loss: 0.1683
Epoch 13, training loss: 0.1567
Epoch 14, training loss: 0.1151
Epoch 15, training loss: 0.1041
Epoch 16, training loss: 0.1024
Epoch 17, training loss: 0

In [5]:
del X_train, y_train

In [6]:
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
test_dataset = MyDataset(X_test, y_test)
from sklearn.metrics import confusion_matrix

# Assuming x_test is a PyTorch tensor
testloader = torch.utils.data.DataLoader(X_test, batch_size=32)
def find_optimal_threshold(predictions, y_test):
    min_sum = float('inf')
    optimal_threshold = 0

    # Iterate over possible thresholds from 0 to 1
    for threshold in np.arange(0.0, 1, 0.001):
        # Apply threshold
        preds = (np.array(predictions)[:,1] > threshold).astype(int)

        # Compute confusion matrix
        cm = confusion_matrix(y_test, preds)

        # Compute sum of off-diagonal elements
        off_diagonal_sum = 164 - np.trace(cm)
        #print(cm)
        # Update optimal threshold if this threshold is better
        if off_diagonal_sum < min_sum and cm[1][1]/np.sum(cm[1]) >0.5:
            min_sum = off_diagonal_sum
            optimal_threshold = threshold

    return optimal_threshold
model.eval()  # Set the model to evaluation mode
predictions = []
output = []
with torch.no_grad():
    for data in testloader:
        outputs = model(data.float().to(device)).to(device)
        predicted = torch.softmax(outputs, dim = 1) # Apply a threshold
        predictions.extend(predicted.tolist())
        output.extend(outputs.tolist())

        
optimal_threshold = find_optimal_threshold(predictions, y_test)
# Calculate the multi-label confusion matrix
mcm = confusion_matrix(y_test, (np.array(predictions)[:,1]>optimal_threshold).astype(int))

print('Confusion Matrix:')
for i, matrix in enumerate(mcm):
    print(f'Class {i}:')
    print(matrix)

Confusion Matrix:
Class 0:
[57 25]
Class 1:
[16 21]


In [7]:

from sklearn.metrics import roc_auc_score

# Convert tensors to numpy arrays for sklearn
outputs_np = (np.array(predictions)[:,1]>optimal_threshold).astype(int)
labels_np = y_test

auc_score = roc_auc_score(labels_np, outputs_np)
print(f"AUC Score: {auc_score}")

AUC Score: 0.6313447593935398
